# Attendance sytem GUI

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import pickle
import pandas as pd

In [2]:
model = tf.keras.models.load_model("upg_student_face_recognition.h5")

In [3]:
with open("labels.pkl" , "rb") as f:
    labels = pickle.load(f)

In [4]:
labels

{'201': 0,
 '202': 1,
 '203': 2,
 '204': 3,
 '205': 4,
 '206': 5,
 '207': 6,
 '208': 7,
 '209': 8,
 '210': 9}

In [8]:
student_mapping = {v:k for k,v in labels.items()}

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
emotion_model = "" # path for the emotion detection model that I have build for the task 1 for visualization activation maps

In [ ]:
start_time = datetime.strptime("09:30:00", "%H:%M:%S").time()
end_time = datetime.strptime("10:00:00", "%H:%M:%S").time()

cap = cv2.VideoCapture(0)

while True:
    now = datetime.now()
    current_time = now.time()
    if current_time < start_time or current_time > end_time:
        print("Attendance system is inactive. Waiting for the allowed time window...")
        time.sleep(10)
        continue

    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    faces = face_cascade.detectMultiScale(rgb, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (128, 128))
        face_normalized = np.expand_dims(face_resized, axis=0) / 255.0
        prediction = model.predict(face_normalized)
        student_id = np.argmax(prediction)
        student_label = student_mapping.get(student_id, 'Unknown')
        emotion_prediction = emotion_model.predict(face_normalized)
        emotion_id = np.argmax(emotion_prediction)
        emotions = ["Happy", "Sad", "Neutral", "Angry", "Surprised"]
        emotion_label = emotions[emotion_id]
        if student_label != "Unknown":
            attendance[student_label] = {
                "Status": "Present",
                "Time": now.strftime("%H:%M:%S"),
                "Emotion": emotion_label
            }

        cv2.putText(frame, f"{student_label} ({emotion_label})", (x, y-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Face Recognition Attendance', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
df = pd.DataFrame.from_dict(attendance, orient="index").reset_index()
df.columns = ["Student", "Status", "Time", "Emotion"]
df.to_excel("attendance.xlsx", index=False)
print("Attendance saved to attendance.xlsx")